In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from sklearn.model_selection import train_test_split


import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Loading Data

In [ ]:
train_data = pd.read_csv("/kaggle/input/digit-recognizer/train.csv")
train_data.shape

In [ ]:
test_data = pd.read_csv("/kaggle/input/digit-recognizer/test.csv")
test_data.shape

In [ ]:
train_data.head()

In [ ]:
test_data.head()

In [ ]:
train_data.isnull().sum().sum()

In [ ]:
plt.figure()
sns.countplot(x=train_data.label)

# Reshape the data
reshape to 28x28 to apply CNN

In [ ]:
# drop target and then reshape remaining data
train_data_2d=train_data.drop('label', axis=1)
train_data_2d=train_data_2d.values.reshape(-1,28,28,1)

test_data_2d=test_data.values.reshape(-1,28,28,1)

In [ ]:
for i in range(5):
    plt.figure(figsize=(1,1))
    plt.imshow(train_data_2d[i])
    plt.show()

In [ ]:
train_data.label[:5]

# Splitting Data

In [ ]:
X = train_data_2d
y = train_data.label

X_test = test_data_2d

In [ ]:
# Normalize the data
X = X/255.0
X_test = X_test/255.0

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)

In [ ]:
train_shape = X_train.shape
train_shape

In [ ]:
y_train.shape

# Modeling

In [ ]:
model = keras.Sequential([
#     CNN base
    layers.Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=(28,28,1), padding='same'),
    layers.MaxPooling2D(2,2),
    
    layers.Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
    layers.MaxPooling2D(2,2),
#     dense head
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation='softmax'),
])

model.summary()

In [ ]:
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'],
)

In [ ]:
history = model.fit(
    X_train, y_train,
    validation_data=(X_valid, y_valid),
#     callbacks=[early_stopping],
    epochs=15,
)

In [ ]:
history_df = pd.DataFrame(history.history)
# Start the plot at epoch 5
history_df.loc[:,['loss', 'val_loss']].plot()
history_df.loc[:, ['accuracy', 'val_accuracy']].plot()

print(("Best Validation Loss: {:0.4f}" +\
      "\nBest Validation Accuracy: {:0.4f}")\
      .format(history_df['val_loss'].min(), 
              history_df['val_accuracy'].max()))

# Confusion Matrix

In [ ]:
pred = model.predict(X_valid)
pred_valid = pd.DataFrame([np.argmax(i) for i in pred])
cm = tf.math.confusion_matrix(labels=y_valid,predictions=pred_valid)
cm

In [ ]:
plt.figure()
sns.heatmap(cm, annot=True,fmt='d')

# Predictions
predictions on test data

In [ ]:
preds = model.predict(X_test)
predictions = [np.argmax(i) for i in preds]
predictions = pd.Series(predictions,name="Label")

In [ ]:
output = pd.concat([pd.Series(range(1,28001),name = "ImageId"),predictions],axis = 1)

output.to_csv("submission.csv",index=False)